#   Dokumentacja klasyfikatora CIFAR-10 używając CNN

Autor: Filip Gębala

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

<h2> Model CNN </h2>
<p>Konwolucyjna sieć neuronowa (CNN) to rodzaj sieci neuronowej typu feedforward, która uczy się istotnych cech danych poprzez optymalizację filtrów (zwanych też jądrami konwolucyjnymi). Tego typu sieci znajdują zastosowanie w różnych dziedzinach — od analizy obrazów, przez przetwarzanie tekstu, aż po dane dźwiękowe. W kontekście analizy obrazów, CNN stały się standardowym rozwiązaniem w zadaniach związanych z widzeniem komputerowym i przetwarzaniem obrazów.</p>

<p>Model oparty jest na czterech warstwach konwolucyjnych, z których każda ma na celu wyodrębnienie coraz bardziej złożonych cech obrazu. Początkowo dane wejściowe (obrazy RGB z CIFAR-10) trafiają do warstwy konwolucyjnej z 64 filtrami, a następnie przechodzą przez normalizację batchową i funkcję aktywacji ReLU. Potem następuje operacja max pooling, która zmniejsza rozmiar przestrzenny cech, zachowując przy tym najważniejsze informacje. Ten schemat jest powtarzany w kolejnych warstwach, z rosnącą liczbą filtrów: 128, 256 i 512.

Po ostatniej warstwie konwolucyjnej zastosowana została warstwa global average pooling, która redukuje każdą mapę cech do jednej wartości, co znacząco zmniejsza liczbę parametrów przed przejściem do warstwy w pełni połączonej. Przed klasyfikatorem dodano jeszcze dropout (z prawdopodobieństwem 0.5), co pomaga zapobiegać przeuczeniu. Ostateczna klasyfikacja odbywa się przez pojedynczą warstwę liniową, która zwraca 10 wartości odpowiadających klasom CIFAR-10. Cały model trenowany jest przy użyciu optymalizatora Adam z regularyzacją wag (weight decay) i harmonogramem uczenia, który stopniowo zmniejsza learning rate, by poprawić stabilność treningu.</p>

In [ ]:
# === MODEL ===
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(2)

        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

<h2>Przygotowanie danych </h2>

<p>Przed rozpoczęciem treningu dane muszą zostać odpowiednio przetworzone. Dla zbioru treningowego zastosowano augmentację danych: obrazy są losowo kadrowane z dodanym paddingiem oraz mogą zostać odbite w poziomie. To sprawia, że sieć widzi więcej różnorodnych wariantów obrazków i lepiej generalizuje. Następnie dane są zamieniane na tensory i znormalizowane — czyli przesunięte i przeskalowane tak, by każda składowa kolorystyczna (R, G, B) miała średnią i odchylenie standardowe odpowiadające wartościom w zbiorze CIFAR-10. Dla danych testowych nie stosuje się augmentacji, tylko sama normalizacja.</p>

In [ ]:
# === TRANSFORMACJE ===
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                        (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                        (0.2023, 0.1994, 0.2010)),
])

<h2>Ładowanie danych oraz inicjacja modelu</h2>

<p>Po wczytaniu danych do loaderów, inicjalizowany jest model oraz optymalizator Adam. Zastosowano też regularyzację L2 (czyli weight decay), by ograniczyć nadmierne dopasowanie modelu do danych treningowych. Do tego dorzucono scheduler, który co 10 epok zmniejsza learning rate o połowę — taki zabieg pomaga przy konwergencji, zwłaszcza w późniejszych etapach uczenia. Funkcja straty to klasyczna entropia krzyżowa, odpowiednia przy klasyfikacji wieloklasowej.</p>

In [ ]:
# === DANE ===
trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# === MODEL I OPT ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

<h2>Trenowanie modelu </h2>

<p>Sam proces uczenia podzielony jest na epoki. W każdej z nich model przechodzi przez wszystkie próbki w zbiorze treningowym. Dla każdej partii obliczane są predykcje, wyznaczana strata, a potem wykonywany jest krok optymalizacji. Po zakończeniu jednej epoki, model oceniany jest na zbiorze testowym. Wynik zapisywany jest w listach train_losses i test_accuracies, żeby można było później narysować wykresy.</p>

In [ ]:
# === TRENING ===
def train_model(model, trainloader, testloader, optimizer, scheduler, criterion, device, epochs):
    train_losses, test_accuracies = [], []
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(trainloader)
        train_losses.append(avg_loss)

        # Ewaluacja
        accuracy = test_model(model, testloader, device)
        test_accuracies.append(accuracy)
        scheduler.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return train_losses, test_accuracies

<h2>Ewaluacja modelu </h2>

<p>Do ewaluacji testowej używana jest oddzielna funkcja test_model, która przełącza model w tryb "eval" — wyłącza on liczenie gradientów i stosowanie dropoutu. Dla każdej partii danych z testu sprawdzana jest poprawność klasyfikacji, a na końcu wyliczana jest całkowita dokładność.</p>

In [ ]:
# === TEST ===
def test_model(model, testloader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

<h2>Wykresy </h2>

<p>Na końcu wykreślane są wykresy — pierwszy pokazuje, jak zmieniała się wartość funkcji straty podczas treningu, drugi przedstawia zmiany dokładności na zbiorze testowym. Dzięki temu łatwo stwierdzić, w którym miejscu model zaczynał się przeuczać</p>

In [ ]:
# === WYKRESY ===
epochs = range(1, len(train_losses)+1)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.title('Strata treningowa')
plt.xlabel('Epoka')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, test_accuracies, label='Test Accuracy', color='orange')
plt.title('Dokładność na zbiorze testowym')
plt.xlabel('Epoka')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()